## Please input your directory for the top level folder
folder name : SUBMISSION MODEL

In [1]:
dir_ = 'E:/Seminararbeit/Code/A1/' # input only here

#### setting other directory

In [2]:
raw_data_dir = dir_+'2. data/'
processed_data_dir = dir_+'2. data/processed/'
log_dir = dir_+'4. logs/'
model_dir = dir_+'5. models/'

In [3]:
####################################################################################
##################### 1-3. recursive model by store & dept #########################
####################################################################################

In [4]:
ver, KKK = 'priv', 0

In [5]:
STORES = ['WI_1', 'WI_2', 'WI_3'] #'CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 
DEPTS = ['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1', 'FOODS_2', 'FOODS_3']

In [6]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

from multiprocessing import Pool

warnings.filterwarnings('ignore')

In [7]:
########################### Helpers
#################################################################################
## Seeder
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [8]:
########################### Helper to load data by store ID
#################################################################################
# Read data
def get_data_by_store(store, dept):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    df = df[df['d']>=START_TRAIN]
    
    df = df[(df['store_id']==store) & (df['dept_id']==dept)]

    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
        
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2
    
    df = pd.concat([df, df3], axis=1)
    del df3
    
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    df = df.reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES:
        for state_id in DEPTS:
            temp_df = pd.read_pickle(processed_data_dir+'test_'+store_id+'_'+state_id+'.pkl')
            temp_df['store_id'] = store_id
            temp_df['dept_id'] = state_id
            base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### Helper to make dynamic rolling lags
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]


def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

In [9]:
########################### Model params
#################################################################################
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.015,
                    'num_leaves': 2**8-1,
                    'min_data_in_leaf': 2**8-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 3000,
                    'boost_from_average': False,
                    'verbose': 1
                } 

In [10]:
########################### Vars
#################################################################################
VER = 1                          
SEED = 42                        
seed_everything(SEED)            
lgb_params['seed'] = SEED        
N_CORES = psutil.cpu_count()     


#LIMITS and const
TARGET      = 'sales'            
START_TRAIN = 700                
END_TRAIN   = 1941 - 28*KKK      
P_HORIZON   = 28                 
USE_AUX     = False             

remove_features = ['id','cat_id', 'state_id','store_id','dept_id',
                   'date','wm_yr_wk','d',TARGET]
mean_features   = ['enc_item_id_store_id_mean','enc_item_id_store_id_std'] 

ORIGINAL = raw_data_dir
BASE     = processed_data_dir+'processedgrid_part_1.pkl'
PRICE    = processed_data_dir+'processedgrid_part_2.pkl'
CALENDAR = processed_data_dir+'processedgrid_part_3.pkl'
LAGS     = processed_data_dir+'processedlags_df_28.pkl'
MEAN_ENC = processed_data_dir+'processedmean_encoding_df.pkl'


#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,30,60]:
        ROLS_SPLIT.append([i,j])

In [11]:
########################### Train Models
#################################################################################
for store_id in STORES:
    for state_id in DEPTS:
        print('Train', store_id, state_id)

        grid_df, features_columns = get_data_by_store(store_id, state_id)

        train_mask = grid_df['d']<=END_TRAIN
        valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
        preds_mask = (grid_df['d']>(END_TRAIN-100)) & (grid_df['d'] <= END_TRAIN+P_HORIZON)

        train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                           label=grid_df[train_mask][TARGET])

        valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                           label=grid_df[valid_mask][TARGET])

        grid_df = grid_df[preds_mask].reset_index(drop=True)
        keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
        grid_df = grid_df[keep_cols]

        d_sales = grid_df[['d','sales']]
        substitute = d_sales['sales'].values
        substitute[(d_sales['d'] > END_TRAIN)] = np.nan
        grid_df['sales'] = substitute

        grid_df.to_pickle(processed_data_dir+'test_'+store_id+'_'+state_id+'.pkl')
        del grid_df, d_sales, substitute

        seed_everything(SEED)
        print('Starte Training:', store_id, state_id)
        estimator = lgb.train(lgb_params,
                              train_data,
                              valid_sets = [valid_data],
                              callbacks=[lgb.log_evaluation(1, True)]
                              
                              )
        
        display(pd.DataFrame({'name':estimator.feature_name(),
                              'imp':estimator.feature_importance()}).sort_values('imp',ascending=False).head(25))
        
        model_name = model_dir+'lgb_model_'+store_id+'_'+state_id+'_v'+str(VER)+'.bin'
        pickle.dump(estimator, open(model_name, 'wb'))

        del train_data, valid_data, estimator
        gc.collect()

        MODEL_FEATURES = features_columns

Train WI_1 HOBBIES_1
Starte Training: WI_1 HOBBIES_1
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.034191 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4738
[LightGBM] [Info] Number of data points in the train set: 489351, number of used features: 66
[1]	valid_0's rmse: 2.14737
[2]	valid_0's rmse: 2.13657
[3]	valid_0's rmse: 2.12618
[4]	valid_0's rmse: 2.1158
[5]	valid_0's rmse: 2.10531
[6]	valid_0's rmse: 2.09498
[7]	valid_0's rmse: 2.08477
[8]	valid_0's rmse: 2.07447
[9]	valid_0's rmse: 2.06432
[10]	valid_0's rmse: 2.05408
[11]	valid_0's rmse: 2.04436
[12]	valid_0's rmse: 2.03471
[13]	valid_0's rmse: 2.0

,name,imp
0,item_id,94263
21,tm_w,23653
20,tm_d,22307
52,rolling_mean_180,19470
54,rolling_mean_tmp_1_7,19308
49,rolling_std_30,18718
53,rolling_std_180,18365
45,rolling_std_7,18255
47,rolling_std_14,17938
51,rolling_std_60,17908


Train WI_1 HOBBIES_2
Starte Training: WI_1 HOBBIES_2
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009553 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3285
[LightGBM] [Info] Number of data points in the train set: 168440, number of used features: 66
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's rmse: 1.04146
[2]	valid_0's rmse: 1.03233
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's rmse: 1.02347
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's rmse: 1.01481
[LightGBM] [Warning] No f

,name,imp
0,item_id,76262
52,rolling_mean_180,36536
53,rolling_std_180,36531
21,tm_w,36031
20,tm_d,35109
51,rolling_std_60,31656
49,rolling_std_30,25241
50,rolling_mean_60,22385
25,tm_dw,21243
65,rolling_mean_tmp_14_60,20500


Train WI_1 HOUSEHOLD_1
Starte Training: WI_1 HOUSEHOLD_1
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027443 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4650
[LightGBM] [Info] Number of data points in the train set: 596596, number of used features: 66
[1]	valid_0's rmse: 1.78301
[2]	valid_0's rmse: 1.77351
[3]	valid_0's rmse: 1.76473
[4]	valid_0's rmse: 1.75591
[5]	valid_0's rmse: 1.74664
[6]	valid_0's rmse: 1.73751
[7]	valid_0's rmse: 1.72878
[8]	valid_0's rmse: 1.71974
[9]	valid_0's rmse: 1.71083
[10]	valid_0's rmse: 1.70238
[11]	valid_0's rmse: 1.69393
[12]	valid_0's rmse: 1.68569
[13]	valid_0's rmse

...

,name,imp
0,item_id,92298
21,tm_w,26288
20,tm_d,25905
52,rolling_mean_180,23160
53,rolling_std_180,21435
49,rolling_std_30,19520
51,rolling_std_60,19427
47,rolling_std_14,18467
50,rolling_mean_60,18352
54,rolling_mean_tmp_1_7,17219


Train WI_3 FOODS_2
Starte Training: WI_3 FOODS_2
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.032477 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4720
[LightGBM] [Info] Number of data points in the train set: 451841, number of used features: 66
[1]	valid_0's rmse: 2.74034
[2]	valid_0's rmse: 2.72582
[3]	valid_0's rmse: 2.71143
[4]	valid_0's rmse: 2.69747
[5]	valid_0's rmse: 2.68258
[6]	valid_0's rmse: 2.66772
[7]	valid_0's rmse: 2.65258
[8]	valid_0's rmse: 2.63747
[9]	valid_0's rmse: 2.62213
[10]	valid_0's rmse: 2.60723
[11]	valid_0's rmse: 2.59176
[12]	valid_0's rmse: 2.57623
[13]	valid_0's rmse: 2.5607

,name,imp
0,item_id,112121
21,tm_w,23445
20,tm_d,22968
52,rolling_mean_180,21799
53,rolling_std_180,19162
51,rolling_std_60,17536
49,rolling_std_30,16890
47,rolling_std_14,16781
11,price_momentum_m,16528
50,rolling_mean_60,16375


Train WI_3 FOODS_3
Starte Training: WI_3 FOODS_3
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.042344 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5906
[LightGBM] [Info] Number of data points in the train set: 932368, number of used features: 66
[1]	valid_0's rmse: 6.42821
[2]	valid_0's rmse: 6.40475
[3]	valid_0's rmse: 6.38032
[4]	valid_0's rmse: 6.35504
[5]	valid_0's rmse: 6.32769
[6]	valid_0's rmse: 6.299
[7]	valid_0's rmse: 6.26916
[8]	valid_0's rmse: 6.23785
[9]	valid_0's rmse: 6.20548
[10]	valid_0's rmse: 6.17266
[11]	valid_0's rmse: 6.13807
[12]	valid_0's rmse: 6.10232
[13]	valid_0's rmse: 6.06544


,name,imp
0,item_id,118371
13,event_name_1,24853
21,tm_w,21009
20,tm_d,20303
54,rolling_mean_tmp_1_7,17600
52,rolling_mean_180,16711
53,rolling_std_180,15877
58,rolling_mean_tmp_7_7,15515
45,rolling_std_7,15498
62,rolling_mean_tmp_14_7,14430
